In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [ ]:
# url = 'https://item.tuhu.cn/Products/TR-JJ-HT5/1.html'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36', 
          'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 
          'accept-encoding': 'gzip, deflate, br', 'accept-language': 'zh-TW,zh;q=0.9,ru;q=0.8,en-US;q=0.7,en;q=0.6,ja;q=0.5', 
          'upgrade-insecure-requests': '1'}


In [ ]:
def get_products():
    links = []
    for i in range(1, 268):
        url = 'https://item.tuhu.cn/Tires/' + str(i) + '/f0.html?ws=home&ln=paneltire&na=alltire'
        res = requests.get(url, headers=headers)
        soup = bs(res.content, 'html.parser')
        table = soup.find('table', class_='List')
        tds = table.find_all('td', class_='td1')
        for t in tds:
            link = t.find('a')['href']
            links.append(link)
        time.sleep(1)
    return links

In [ ]:
links = get_products()

In [ ]:
def parse(links):
    contents = []
    for link in links:
        res = requests.get(url=link, headers=headers)
        soup = bs(res.content, 'html.parser')
        properties = soup.find('div', class_='properties').find_all('li')
        keys = []
        values = []
        for p in properties:
            text = p.text.split('：')
            keys.append(text[0])
            values.append(text[1])
        data = dict(zip(keys, values))

        price_tag = soup.find('div', class_='price')
        price = price_tag.text.strip()
        data.setdefault('Price', price)
        contents.append(data)
    return contents

In [ ]:
contents = parse(links)

In [ ]:
contents

In [ ]:
import concurrent.futures

In [ ]:
def parse_item(link):
    res = requests.get(url=link, headers=headers)
    soup = bs(res.content, 'html.parser')
    properties = soup.find('div', class_='properties').find_all('li')
    keys = []
    values = []
    for p in properties:
        text = p.text.split('：')
        keys.append(text[0])
        values.append(text[1])
    data = dict(zip(keys, values))

    price_tag = soup.find('div', class_='price')
    price = price_tag.text.strip()
    data.setdefault('Price', price)
    return data

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    results = list(executor.map(parse_item, links))
    

In [ ]:
df = pd.DataFrame()
for result in results:
    df = df.append(result, ignore_index=True)
df

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.to_excel('tuhu_tires.xlsx', index=False)